In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("santander/train.csv")
test = pd.read_csv("santander/test.csv")
sample = pd.read_csv("santander/sample_submission.csv")

In [5]:
display(train.head())
display(sample.head())

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [6]:
print(train.shape,test.shape)
full = pd.concat([train.drop("target",axis=1),test])
full = full.drop("ID_code",axis=1)
display(full.info())

(200000, 202) (200000, 201)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 0 to 199999
Columns: 200 entries, var_0 to var_199
dtypes: float64(200)
memory usage: 613.4 MB


None

In [12]:
from sklearn.model_selection import train_test_split
X = train.iloc[:,2:]
y = train.iloc[:,1]
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.5,random_state=0)

In [13]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

'''
print("xgb score")
clf = xgb.XGBClassifier(max_depth=5, n_estimators=200, learning_rate=0.05)
model = cross_validate(clf,train_X,train_y,cv=2,return_train_score=True)
print(model["train_score"].mean(),model["train_score"].std())
print(model["test_score"].mean(),model["test_score"].std())
clf.fit(train_X,train_y)
print("testscore:",clf.score(test_X,test_y))
'''

print("RandomForest score")
clf = RandomForestClassifier(max_depth=5, n_estimators=200)
model = cross_validate(clf,train_X,train_y,cv=2,return_train_score=True)
print(model["train_score"].mean(),model["train_score"].std())
print(model["test_score"].mean(),model["test_score"].std())

clf.fit(train_X,train_y)
print("testscore:",clf.score(test_X,test_y))

RandomForest score
0.899790000159916 7.995800003157694e-06
0.899790000159916 7.995800003157694e-06
testscore: 0.89923


In [15]:
model["test_score"]

array([0.899782, 0.899798])

In [17]:
pred=clf.predict(test.iloc[:,1:])
submission = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": pred
    })
submission.head()
submission.to_csv('submission.csv', index=False)

In [27]:
train["target"].sum()/len(train)

0.10049

In [29]:
train.iloc[:,2:].shape

(200000, 200)

In [31]:
test.iloc[:,1:].shape

(200000, 200)

In [65]:
train1 = train[train["target"]==1]
train0 = train[train["target"]==0].sample(n=20000)
train_s = pd.concat([train1,train0])
train_s = train_s.sample(frac=1)
train_s.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
196051,train_196051,1,13.2550,6.2935,14.5240,7.4259,10.3731,-12.4433,5.7332,18.5987,...,-0.1488,12.6130,0.2644,5.1511,17.4276,2.3515,-5.7334,10.2903,15.1350,11.4468
20138,train_20138,1,13.9009,-4.9679,11.3716,6.5470,12.1373,5.6846,4.8031,16.8994,...,0.9810,9.3569,0.5672,1.4267,22.7532,1.4910,6.3687,8.7561,16.4306,-13.9519
9191,train_9191,0,6.6170,2.1974,13.1309,7.2332,10.6548,-1.9190,5.7056,12.6975,...,0.3371,11.0356,1.4780,3.6111,19.1937,0.4218,-5.0395,10.4211,19.4929,-2.8482
163020,train_163020,1,14.0884,-5.0170,7.6302,5.5834,11.6152,-3.3844,4.7681,21.1267,...,-4.7785,13.0373,1.3055,4.7254,13.8450,-0.6965,4.9384,8.1010,18.3330,-4.7157
120898,train_120898,1,7.4291,2.6190,13.6420,8.8007,12.6953,-5.4826,4.7049,16.6292,...,0.5248,9.6973,-0.2276,3.5983,22.3815,-0.7096,4.1032,9.3116,17.0245,11.2197


In [66]:
X = train_s.iloc[:,2:]
y = train_s.iloc[:,1]
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.5,random_state=0)

In [69]:
pred=clf.predict(test.iloc[:,1:])
submission = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": pred
    })
submission.head()
submission.to_csv('submission.csv', index=False)

In [68]:
for max_depth in [3]:
    for n in [200,300]:
        for lr in [0.1,0.2]:
            clf = xgb.XGBClassifier(max_depth=max_depth, n_estimators=n, learning_rate=lr)
            print("max_depth=",max_depth,"n_estimatiors=",n,"learning_rate=",lr)
            clf.fit(train_X,train_y)
            print("trainscore:",clf.score(train_X,train_y))
            print("testscore:",clf.score(test_X,test_y))

max_depth= 3 n_estimatiors= 200 learning_rate= 0.1
trainscore: 0.8512644022145743
testscore: 0.771360167589406
max_depth= 3 n_estimatiors= 200 learning_rate= 0.2
trainscore: 0.8919646865180308
testscore: 0.7840790064342361
max_depth= 3 n_estimatiors= 300 learning_rate= 0.1
trainscore: 0.8783480472841538
testscore: 0.7826824280512744
max_depth= 3 n_estimatiors= 300 learning_rate= 0.2
trainscore: 0.9176517532046486
testscore: 0.7929572547259215


In [61]:
cor = train_s.corr()
i = cor["target"][(cor["target"]>0.07) | (cor["target"]<-0.07)].index
train_s = train_s[i]
X = train_s.iloc[:,1:]
y = train_s.iloc[:,0]
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.5,random_state=0)

In [62]:
print("RandomForest score")
clf = RandomForestClassifier(max_depth=5, n_estimators=500)
model = cross_validate(clf,train_X,train_y,cv=5,return_train_score=True)
print(model["train_score"].mean(),model["train_score"].std())
print(model["test_score"].mean(),model["test_score"].std())

clf.fit(train_X,train_y)
print("testscore:",clf.score(test_X,test_y))

RandomForest score
0.7260462383375643 0.001915984783584825
0.686318319532176 0.009311344280469307
testscore: 0.6851214524415182
